In [ ]:
import pandas as pd


df = pd.read_json('final_data/HA_ALL_nonllm.json', orient='records', lines=True)

df

In [ ]:
# for rows with null value in their review_suggestion, put the value as follows: review_rating < 4: reject, review_rating >= 4 and < 6: revision, review_rating >= 6: accept
def fill_review_suggestion(row):
    if row['review_suggestion'] is None:
        if row['review_rating'] < 4:
            return 'Reject'
        elif 4 <= row['review_rating'] < 6:
            return 'Revision'
        elif row['review_rating'] >= 6:
            return 'Accept'
    else:
        return row['review_suggestion']
df['review_suggestion'] = df.apply(fill_review_suggestion, axis=1)
df

In [ ]:
# in review_suggestion, replace Approved With Reservations with Revision, Approved with Accept, Not Approved with Reject
def replace_review_suggestion(row):
    if row['review_suggestion'] == 'Approved With Reservations':
        return 'Revision'
    elif row['review_suggestion'] == 'Approved':
        return 'Accept'
    elif row['review_suggestion'] == 'Not Approved':
        return 'Reject'
    else:
        return row['review_suggestion']
df['review_suggestion'] = df.apply(replace_review_suggestion, axis=1)
df

In [ ]:
# in review_suggestion, replace Major Revision and Minor Revision with Revision
def replace_review_suggestion(row):
    if row['review_suggestion'] == 'Major Revision':
        return 'Revision'
    elif row['review_suggestion'] == 'Minor Revision':
        return 'Revision'
    else:
        return row['review_suggestion']
df['review_suggestion'] = df.apply(replace_review_suggestion, axis=1)
df

In [ ]:
# in review_suggestion, replace any value that has accept term in it with Accept and any value that has reject term in it with Reject
def replace_review_suggestion(row):
    if 'accept' in row['review_suggestion'].lower():
        return 'Accept'
    elif 'reject' in row['review_suggestion'].lower():
        return 'Reject'
    else:
        return row['review_suggestion']
df['review_suggestion'] = df.apply(replace_review_suggestion, axis=1)
df

In [ ]:
# drop all columns except for review_suggestion, paper_id, reviewer, venue
df_decision = df[['paper_id', 'review_suggestion', 'reviewer', 'venue']]
df_decision

In [ ]:
# in df_decision['reviewer], replace all ' ' and '_' with '-'
def replace_reviewer(row):
    return row['reviewer'].replace(' ', '-').replace('_', '-')
df_decision['reviewer'] = df_decision.apply(replace_reviewer, axis=1)
df_decision

In [ ]:
import os
import glob
import json
import pandas as pd


def load_reviews(folder_path):
    rows = []
    # find all JSON files in the folder
    for file_path in glob.glob(os.path.join(folder_path, '*.json')):
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        paper_id = data.get('paper_id')
        assessor = data.get('assessor')
        metrics = data.get('metrics', {})
        
        # group metrics by reviewer name
        reviewer_metrics = {}
        for key, value in metrics.items():
            # only process keys that start with "review_"
            if not key.startswith('review_'):
                continue
            parts = key.split('_')
            reviewer = parts[1]                          # e.g. "Palwinder-Singh"
            metric_name = '_'.join(parts[2:])            # e.g. "Comprehensiveness"
            
            reviewer_metrics.setdefault(reviewer, {})
            reviewer_metrics[reviewer][metric_name] = value
        
        # turn each reviewer’s metrics into a row
        for reviewer, mdict in reviewer_metrics.items():
            row = {
                'paper_id': paper_id,
                'assessor': assessor,
                'reviewer': reviewer
            }
            row.update(mdict)
            rows.append(row)
    
    # build the final DataFrame
    df = pd.DataFrame(rows)
    return df

# Example usage:
folder = 'Human_Annotation_Data'
df_human = load_reviews(folder)

# show the first few rows
df_human = df_human[df_human['Overall_Quality'] > 10]
df_human

In [ ]:
df_human = df_human[['paper_id', 'reviewer', 'assessor', 'Overall_Quality']]
df_human

In [ ]:
# merge df_human and df_decision on paper_id and reviewer
df_human['paper_id'] = df_human['paper_id'].astype(int)
df_decision['paper_id'] = df_decision['paper_id'].astype(int)
df_human['reviewer'] = df_human['reviewer'].astype(str)
df_decision['reviewer'] = df_decision['reviewer'].astype(str)

df_merge = pd.merge(df_human, df_decision, on=['paper_id', 'reviewer'], how='inner')
df_merge

In [25]:
df_iclr = df_merge[df_merge['venue'] == 'iclr']
df_neurips = df_merge[df_merge['venue'] == 'neurips']
df_sw = df_merge[df_merge['venue'] == 'semanticweb']
df_f1000 = df_merge[df_merge['venue'] == 'f1000']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# color map for suggestions/decisions
color_map = {
    'Accept': 'green',
    'Reject': 'red',
    'Revision': 'blue'
}

# Define a function to plot politeness_score per decision distribution
def plot_distribution(df, title):
    # Add a decision column based on review_rating [1, 4) reject, [4, 6) revision, [6, 10] accept
    # df['decision'] = pd.cut(df['Overall_Quality'], bins=[0, 4, 6, 10], labels=['Reject', 'Revision', 'Accept'], right=False)
    
    # Plot politeness_score distribution
    plt.figure(figsize=(10, 6))
    for decision in df['review_suggestion'].unique():
        sns.kdeplot(
            data=df[df['review_suggestion'] == decision],
            x='Overall_Quality',
            label=decision,
            color=color_map.get(decision, 'black'),
            fill=False,
            alpha=0.9,
            linewidth=3,
            cut=0
        )
    plt.title(f"Overall Quality Score Distribution - {title}", fontsize=16)
    plt.xlabel("Overall Quality Score", fontsize=12)
    plt.ylabel("Density", fontsize=12)
    plt.legend(title="", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Plot

plot_distribution(df_merge, 'Human Annotation')

# for df in [df_iclr, df_neurips, df_sw, df_f1000]:
#     plot_distribution(df, 'Human Annotation')